In [146]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.openai import OpenAI
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [147]:
class RAG_agent:
    def __init__(self, index_path):
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        index = load_index_from_storage(storage_context)
        llm = OpenAI(
            model="gpt-4o",
            temperature=0.7,
            top_p=0.95,
            frequency_penalty=0.2,
            num_outputs=1000,
        )
        self.query_engine = index.as_query_engine(response_mode="refine", similarity_top_k=5, verbose = True, llm = llm)

    def query(self, query):
        return self.query_engine.query(query)


In [148]:
class Query_agent:
    # This agent takes user prompt and refine to a query for specific RAG agent
    def __init__(self, description, history):
        self.client = openai.OpenAI()
        self.history = history
        self.description = [
            {"role": "system", "content": description},
        ]

    def create_query(self):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.history + self.description,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content
        

In [149]:
class Response_agent:
    # This agent takes the history and create the final response to user
    def __init__(self, description, history):
        self.client = openai.OpenAI()
        self.messages = history
        self.description = [
            {"role": "system", "content": description},
        ]

    def respond(self):
        response = self.client.chat.completions.create(
            model="gpt-4o",
            messages=self.messages + self.description,
            max_tokens=1000,
            n=1,
            top_p=0.95,
            frequency_penalty=0.2,
            temperature=0.7
        )
        return response.choices[0].message.content

In [150]:
def load_prompt(filename):
    with open(f'agent_descriptions/{filename}.txt', 'r') as file:
        return file.read().strip()

STRATEGY_QUERY_DESCRIPTION = load_prompt('strategy_agent')
GAMEINFO_QUERY_DESCRIPTION = load_prompt('gameinfo_agent')
FINAL_RESPONSE_DESCRIPTION = load_prompt('final_agent')
LOOP_RESPONSE_DESCRIPTION = load_prompt('loop_agent')

In [151]:
KEYWORD = "ADDITIONAL_QUERY_REQUIRED"

In [152]:
class EldenGuideSystem:
    # The agent system that takes user prompt and return the final response
    def __init__(self, strategy_agent, gameinfo_agent, 
                 strategy_query_description, 
                 gameinfo_query_description, 
                 loop_response_description, 
                 final_response_description):
        self.messages = []
        self.strategy_agent = strategy_agent
        self.gameinfo_agent = gameinfo_agent
        self.strategy_query_agent = Query_agent(strategy_query_description, self.messages)
        self.gameinfo_query_agent = Query_agent(gameinfo_query_description, self.messages)
        self.loop_response_agent = Response_agent(loop_response_description, self.messages)
        self.final_response_agent = Response_agent(final_response_description, self.messages)

    def push_assistant_message(self, message):
        self.messages.append({
            "role": "assistant",
            "content": message
        })

    def run(self, prompt):
        count = 0
        self.messages.append({"role": "user", "content": prompt})
        while count < 3:
            print(f"Querying... {count}")
            strategy_query = self.strategy_query_agent.create_query()
            strategy_response = self.strategy_agent.query(strategy_query)
            self.push_assistant_message("Strategy query result: " + str(strategy_response))
            gameinfo_query = self.gameinfo_query_agent.create_query()
            gameinfo_response = self.gameinfo_agent.query(gameinfo_query)
            self.push_assistant_message("In-game data query result: " + str(gameinfo_response))
            loop_response = self.loop_response_agent.respond()
            if KEYWORD in loop_response:
                count += 1
            else:
                break

        if count >= 3:
            final_response = self.final_response_agent.respond()
        else:
            final_response = loop_response
        return final_response


In [94]:
# index paths
strategy_path = "index/strategy"
gameinfo_path = "index/game"
altogether_path = "index/altogether"

In [35]:
strategy_agent = RAG_agent(strategy_path)
gameinfo_agent = RAG_agent(gameinfo_path)

In [156]:
prompt = "Nagakiba belongs to which NPC?"

In [157]:
system = EldenGuideSystem(strategy_agent, gameinfo_agent,
                          STRATEGY_QUERY_DESCRIPTION,
                          GAMEINFO_QUERY_DESCRIPTION,
                          LOOP_RESPONSE_DESCRIPTION,
                          FINAL_RESPONSE_DESCRIPTION)
response = system.run(prompt)
print(response)

Querying... 0
In Elden Ring, the Nagakiba is associated with the NPC Yura, Hunter of Bloody Fingers. You can obtain the Nagakiba by progressing through Yura's questline, which involves assisting him in fighting against Bloody Finger invaders. Alternatively, if Yura is defeated at any point, the Nagakiba can be collected from his location.


In [159]:
for message in system.messages:
    print(message)


{'role': 'user', 'content': 'Nagakiba belongs to which NPC?'}
{'role': 'assistant', 'content': 'Strategy query result: To create an effective weapon strategy build for the Nagakiba in Elden Ring, utilize its long reach and versatility by boosting Dexterity and Strength to maximize damage. Infuse the weapon with Ashes of War that align with your combat style, whether aggressive or tactical. Enhance the build further by using talismans or armor that increase Dexterity or overall attack power.'}
{'role': 'assistant', 'content': "In-game data query result: I'm sorry, the information regarding the location of the Nagakiba weapon is not available."}
